<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Часть 2
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План 2 части домашнего задания
    2.1. Введение
    2.2. Описание данных
    2.3. Предобработка данных
    2.4. Обучение и проверка моделей
    2.5. Заключение

### 2.1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://goo.gl/forms/z8zENbMiaEAeB7nG3) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2.2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://yadi.sk/d/krikdUic3Ggjyf) эту выборку. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 stack.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l stack.tsv

10000000 stack.tsv
CPU times: user 1.01 s, sys: 132 ms, total: 1.14 s
Wall time: 25.3 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 2.3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [5]:
import os
#from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

In [4]:
!python preprocess.py stackoverflow.10kk.tsv stackoverflow.vw

10000000it [01:20, 123690.31it/s]
4389054 lines selected, 15 lines corrupted.


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [ ]:
''' ВАШ КОД ЗДЕСЬ '''
#!split -l 1463018 stackoverflow.vw stackoverflow
#mv all files

### 2.4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [15]:
''' ВАШ КОД ЗДЕСЬ '''
!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 1 --passes 1 -f stacktrain_model.vw --quiet

In [4]:
#validate
!vw -i stacktrain_model.vw -t -d stackoverflow_valid.vw -p valid_1_1.txt --quiet

In [ ]:
!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 2 --passes 1 -f stacktrain_model12.vw --quiet
#!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 3 --passes 1 -f stacktrain_model13.vw --quiet
#!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 1 --passes 2 -f stacktrain_model21.vw --quiet
#!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 2 --passes 2 -f stacktrain_model22.vw --quiet
#!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 3 --passes 2 -f stacktrain_model23.vw --quiet
#!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 1 --passes 3 -f stacktrain_model31.vw --quiet
#!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 2 --passes 3 -f stacktrain_model32.vw --quiet
#!vw -d stackoverflow_train.vw --oaa 10 --random_seed 17 -b 28 --ngram 3 --passes 3 -f stacktrain_model33.vw --quiet

In [ ]:
!vw -i stacktrain_model12.vw -t -d stackoverflow_valid.vw -p valid_1_2.txt --quiet
!vw -i stacktrain_model13.vw -t -d stackoverflow_valid.vw -p valid_1_3.txt --quiet
!vw -i stacktrain_model21.vw -t -d stackoverflow_valid.vw -p valid_2_1.txt --quiet
!vw -i stacktrain_model22.vw -t -d stackoverflow_valid.vw -p valid_2_2.txt --quiet
!vw -i stacktrain_model23.vw -t -d stackoverflow_valid.vw -p valid_2_3.txt --quiet
!vw -i stacktrain_model31.vw -t -d stackoverflow_valid.vw -p valid_3_1.txt --quiet
!vw -i stacktrain_model32.vw -t -d stackoverflow_valid.vw -p valid_3_2.txt --quiet
!vw -i stacktrain_model33.vw -t -d stackoverflow_valid.vw -p valid_1_3.txt --quiet

In [14]:
%%time
target_valid = []
target_test = []

lines = 0
with open("stackoverflow_valid.vw") as valid_file:
    for line in valid_file:
        lines+=1
        splitted_line = line.split('|')
        target_valid.append(splitted_line[0].strip())
print(lines, "valid target lines") 

lines = 0
with open("stackoverflow_test.vw") as test_file:
    for line in test_file:
        lines+=1
        splitted_line = line.split('|')
        target_test.append(splitted_line[0].strip())
print(lines, "test target lines")        

1463018 valid target lines
1463018 test target lines
CPU times: user 11.2 s, sys: 960 ms, total: 12.2 s
Wall time: 12.2 s


In [15]:
target_valid = [int(i) for i in target_valid]
target_test = [int(i) for i in target_test]

In [16]:
target_valid[:10]

[2, 7, 5, 5, 6, 1, 7, 7, 7, 4]

In [18]:
best_accuracy_score = 0
best_accuracy_name = ''
for pas in [1,3,5]:
    for ngr in [1,2,3]:
        filename = 'valid_'+ str(pas) +'_'+ str(ngr) +'.txt'
        with open(filename) as pred_file:
            valid_prediction = [int(label) 
                             for label in pred_file.readlines()]
            print(filename)
            #print(valid_prediction[:10])
            #print(target_valid[:10])
            accuracy = accuracy_score(target_valid, valid_prediction)
            if accuracy > best_accuracy_score:
                best_accuracy_score = accuracy
                best_accuracy_name = filename
            print("Accuracy for " + filename + " : {}".format(accuracy))

print("Best accuracy is  ", best_accuracy_name)
print("Best accuracy score is ", best_accuracy_score)
#print("AUC: {}".format(round(roc_auc_score(target_valid, valid_prediction), 3)))

valid_1_1.txt
Accuracy for valid_1_1.txt : 0.9151849122840594
valid_1_2.txt
Accuracy for valid_1_2.txt : 0.9310705678262332
valid_1_3.txt
Accuracy for valid_1_3.txt : 0.32782166726588463
valid_3_1.txt
Accuracy for valid_3_1.txt : 0.9143797273854457
valid_3_2.txt
Accuracy for valid_3_2.txt : 0.9278320567484474
valid_3_3.txt
Accuracy for valid_3_3.txt : 0.9264240084537579
valid_5_1.txt
Accuracy for valid_5_1.txt : 0.9134077639509561
valid_5_2.txt
Accuracy for valid_5_2.txt : 0.9292154983739093
valid_5_3.txt
Accuracy for valid_5_3.txt : 0.9261902450960959
Best accuracy is   valid_1_2.txt
Best accuracy score is  0.931070567826


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке - этот
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [21]:
''' ВАШ КОД ЗДЕСЬ '''
#console - vw -i stacktrain_model12.vw -t -d stackoverflow_test.vw -p test_1_2.txt

filename = 'test_1_2.txt'
test_accuracy_1_2 = 0
with open(filename) as pred_file:
    valid_prediction = [int(label) 
        for label in pred_file.readlines()]
    test_accuracy_1_2 = accuracy_score(target_test, valid_prediction)
print("Accuracy for test" + " : {}".format(test_accuracy_1_2))

Accuracy for test : 0.93117582968904


In [22]:
test_accuracy_1_2 - best_accuracy_score

0.00010526186280679362

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5% - этот. на тесте вообще вышло лучше о_О

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [23]:
''' ВАШ КОД ЗДЕСЬ '''
#обучаем
#console - vw -d stackoverflow_uni.vw --oaa 10 --random_seed 17 -b 28 --ngram 2 --passes 1 -f stackuni_model.vw

#предсказываем для valid - на всякий случай
#console - vw -i stackuni_model.vw -t -d stackoverflow_valid.vw -p valid_uni.txt

filename = 'valid_uni.txt'
uni_valid_accuracy = 0
with open(filename) as pred_file:
    valid_prediction = [int(label) 
        for label in pred_file.readlines()]
    uni_valid_accuracy = accuracy_score(target_valid, valid_prediction)
print("Accuracy for uni" + " : {}".format(uni_valid_accuracy))

Accuracy for uni : 0.9869810214228396


In [25]:
#прескажем для тест
#console - vw -i stackuni_model.vw -t -d stackoverflow_test.vw -p test_uni.txt

filename = 'test_uni.txt'
uni_test_accuracy = 0
with open(filename) as pred_file:
    valid_prediction = [int(label) 
        for label in pred_file.readlines()]
    uni_test_accuracy = accuracy_score(target_test, valid_prediction)
print("Accuracy for uni" + " : {}".format(uni_test_accuracy))

Accuracy for uni : 0.9352892445615844


In [26]:
uni_test_accuracy - test_accuracy_1_2

0.004113414872544352

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4% - этот
 - 0.8%
 - 1.2%

### 2.5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)